# **Horse Colic XGB Model**

This notebook intend to predict the status of the horse after some features presents in this data set. 
In this script i will use some feature engineering and data preprocessing as category transformation
and imputing through null values.

PS: My english is not good 😁

## The basic import automated by Kaggle

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Loading the dataset in to Pandas DataFrame

In [ ]:
horse_data = pd.read_csv("/kaggle/input/horse-colic/horse.csv")
horse_data

# Checking the types presents in dataset

In [ ]:
horse_data.dtypes

## Checking nulls and empty values

In [ ]:
display(pd.DataFrame(horse_data.isna().sum()))

## Encoding the categorics attributes

In [ ]:
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder

encoder = LabelEncoder()

y = pd.DataFrame({'outcome' : encoder.fit_transform(horse_data['outcome'])})

encoder1 = TargetEncoder(handle_missing='return_nan',return_df=True)

horse_enc = encoder1.fit_transform(horse_data,y)

horse_enc

## Making the imputation

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5,weights='uniform')

horse_enc = pd.DataFrame(imputer.fit_transform(horse_enc),columns=horse_data.columns)
    
horse_enc['outcome'] = horse_enc['outcome'].astype(np.uint8)
horse_enc.dtypes

## Seeing the correlation between the target and instances

In [ ]:
display(horse_enc.corr().abs()['outcome'].sort_values())

## Preparing the model

After looking the correlation numbers and some experiments, 6 attributes was ignored due to high correlation or was too low.

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = horse_enc.drop(['nasogastric_reflux_ph','outcome','rectal_temp','nasogastric_tube','lesion_3','pain'],axis=1)
target = horse_enc['outcome']

X_train, X_test, y_train, y_test = train_test_split(data,target, random_state=2020)

xgb = XGBClassifier(n_estimators=1000,learning_rate=0.025,random_state=2020)

xgb.fit(X_train,y_train,early_stopping_rounds=10,eval_set=[(X_test,y_test)])

## The accuracy score

In [ ]:
y_pred = xgb.predict(X_test)

print('Accuracy: ',accuracy_score(y_test, y_pred))

Feel free to give a upvote and suggestion are welcome!